## Import Libraries

In [1]:
# standard library imports
from ast import literal_eval
import itertools
import time
import re

# third-party imports
import numpy as np
import pandas as pd

# customisations
pd.set_option("display.max_columns", 100)

# SteamApp Data 
Importing and Cleaning SteamApp Data 
- `Step 1:` Identify Null Values & Missing Values
- `Step 2:` Removing Columns that are irrelevant for answering our question
- `Step 3:` Changing the way data is represented in certain columns
- `Step 4:` Exporting Cleaned SteamApp DataSet

In [2]:
# Import SteamApp data
raw_steam_data = pd.read_csv('Uncleaned\steam_app_data.csv')

# print out number of rows and columns
print('Rows:', raw_steam_data.shape[0])
print('Columns:', raw_steam_data.shape[1])

# view first five rows
raw_steam_data.head()

Rows: 71193
Columns: 31


C:\Users\zongh\AppData\Local\Temp\ipykernel_3664\1970854512.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_steam_data = pd.read_csv('Uncleaned\steam_app_data.csv')


,type,name,steam_appid,required_age,is_free,controller_support,dlc,fullgame,supported_languages,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,recommendations,achievements,release_date,support_info,ratings
0,game,Counter-Strike,10,0.0,False,NaN,NaN,NaN,"English<strong>*</strong>, French<strong>*</st...",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'SGD', 'initial': 1000, 'final': ...","[574941, 7]","[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 148406},NaN,"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",{'usk': {'rating': '16'}}
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,NaN,"English, French, German, Italian, Spanish - Sp...",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'SGD', 'initial': 525, 'final': 5...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 5981},NaN,"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",NaN
2,game,Day of Defeat,30,0.0,False,NaN,NaN,NaN,"English, French, German, Italian, Spanish - Spain",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'SGD', 'initial': 525, 'final': 5...","[30, 944613]","[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 3984},NaN,"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",{'usk': {'rating': '16'}}
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,NaN,"English, French, German, Italian, Spanish - Sp...",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'SGD', 'initial': 525, 'final': 5...",[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 2093},NaN,"{'coming_soon': False, 'date': '1 Jun, 2001'}","{'url': '', 'email': ''}",NaN
4,game,Half-Life: Opposing Force,50,0.0,False,NaN,NaN,NaN,"English, French, German, Korean",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Gearbox Software'],['Valve'],NaN,"{'currency': 'SGD', 'initial': 525, 'final': 5...",[32],"[{'name': 'default', 'title': 'Buy Half-Life: ...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",{'total': 18597},NaN,"{'coming_soon': False, 'date': '1 Nov, 1999'}","{'url': 'https://help.steampowered.com', 'emai...",NaN


In [3]:
# Data Types in SteamApp
raw_steam_data.dtypes

type                        object
name                        object
steam_appid                  int64
required_age                object
is_free                     object
controller_support          object
dlc                         object
fullgame                   float64
supported_languages         object
pc_requirements             object
mac_requirements            object
linux_requirements          object
legal_notice                object
drm_notice                  object
ext_user_account_notice     object
developers                  object
publishers                  object
demos                       object
price_overview              object
packages                    object
package_groups              object
platforms                   object
metacritic                  object
reviews                     object
categories                  object
genres                      object
recommendations             object
achievements                object
release_date        

## 1. Handling missing Data
- The initial data appears to have a mix of numeric and text columns, along with many missing values. Some columns even seem to contain dictionaries or lists.
- To identify problematic missing value counts, we can use chaining with the `isnull()` and `sum()` methods. This quickly reveals columns with over **20,000 missing values** in a 30,000 row dataset, suggesting they likely hold little value.

In [4]:
null_counts = raw_steam_data.isnull().sum()
null_counts

type                         235
name                          13
steam_appid                    0
required_age                 235
is_free                      235
controller_support         54999
dlc                        58944
fullgame                   71193
supported_languages          274
pc_requirements              235
mac_requirements             235
linux_requirements           235
legal_notice               48780
drm_notice                 70822
ext_user_account_notice    70056
developers                   425
publishers                   235
demos                      64189
price_overview             10705
packages                   10032
package_groups               235
platforms                    235
metacritic                 66720
reviews                    60235
categories                  1104
genres                       384
recommendations            54175
achievements               30941
release_date                 235
support_info                 235
ratings   

- Since handling each column might be necessary, we'll define functions for an organized approach. 
- The below function tackles columns with high missing values **(>50%)**. We'll use a DataFrame filter to remove these columns efficiently.

In [5]:
threshold = raw_steam_data.shape[0] // 2

print('Drop columns with more than {} missing rows'.format(threshold))
print()

drop_rows = raw_steam_data.columns[null_counts > threshold]

print('Columns to drop: {}'.format(list(drop_rows)))

Drop columns with more than 35596 missing rows

Columns to drop: ['controller_support', 'dlc', 'fullgame', 'legal_notice', 'drm_notice', 'ext_user_account_notice', 'demos', 'metacritic', 'reviews', 'recommendations']


## 1.1 Handling Column Type
We filtered through the `type` column and found 2 strings under the column: "game" and "NaN"
- Using isnul()
- Using value_counts() <br>

Through this we found 235 rows containing the NaN type. We will be removing these rows later on as these rows are unlikely to be games thus irrelavant in answering our main question.

In [6]:
print('Rows to remove:', raw_steam_data[raw_steam_data['type'].isnull()].shape[0])

# preview rows with missing type data
raw_steam_data[raw_steam_data['type'].isnull()].head(3)

Rows to remove: 235


,type,name,steam_appid,required_age,is_free,controller_support,dlc,fullgame,supported_languages,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,recommendations,achievements,release_date,support_info,ratings
26,NaN,ValveTestApp852,852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,NaN,Star Trek: DAC - Demo,4330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,NaN,Puzzlegeddon,8740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
raw_steam_data['type'].value_counts(dropna=False)

type
game    70958
NaN       235
Name: count, dtype: int64

## 1.2 Handling Column Name
Filtering through the `name` column, we found that there are 3 different types of strings. This combination is achieved using boolean filters with brackets and the | symbol (logical OR). Hence, removing them may be a good idea.

1. The name of the game (including non-english)
2. "none" string
3. NaN value <br>

We then decided to remove the "none" strings and NaN value under `name` column as the data in their rows seemed to contain NaN values as well, proving irrelavant in answering our question 

In [8]:
print('Rows to remove:', raw_steam_data[(raw_steam_data['name'].isnull()) | (raw_steam_data['name'] == 'none')].shape[0])
raw_steam_data[(raw_steam_data['name'].isnull()) | (raw_steam_data['name'] == 'none')].head(3)

Rows to remove: 15


,type,name,steam_appid,required_age,is_free,controller_support,dlc,fullgame,supported_languages,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,recommendations,achievements,release_date,support_info,ratings
486,NaN,NaN,17760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5017,game,none,339860,0.0,False,NaN,NaN,NaN,English<strong>*</strong><br><strong>*</strong...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],NaN,NaN,NaN,NaN,[''],NaN,NaN,NaN,[],"{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...",NaN,"{'total': 3, 'highlighted': [{'name': 'Master ...","{'coming_soon': False, 'date': '27 Feb, 2015'}","{'url': '', 'email': ''}",NaN
7443,game,none,398970,0.0,False,NaN,NaN,NaN,English<strong>*</strong><br><strong>*</strong...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,NaN,NaN,['none'],['none'],"[{'appid': 516340, 'description': ''}]",NaN,NaN,[],"{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...",NaN,"{'total': 35, 'highlighted': [{'name': ""They'v...","{'coming_soon': False, 'date': '5 Nov, 2015'}","{'url': '', 'email': ''}",NaN


## 1.3 Handling Duplicate AppIDs
We'll use the DataFrame.duplicated() method to identify and remove duplicate rows based on the "AppID" column. This ensures each game has a unique identifier in our dataset.

In [9]:
duplicate_rows = raw_steam_data[raw_steam_data.duplicated()]

print('Duplicate rows to remove:', duplicate_rows.shape[0])

duplicate_rows.head(3)

Duplicate rows to remove: 6212


,type,name,steam_appid,required_age,is_free,controller_support,dlc,fullgame,supported_languages,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,recommendations,achievements,release_date,support_info,ratings
31,game,SiN Episodes: Emergence,1300,0.0,False,NaN,NaN,NaN,"English, Russian, French",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],NaN,NaN,NaN,['Ritual Entertainment'],['Ritual Entertainment'],NaN,"{'currency': 'SGD', 'initial': 1000, 'final': ...",[443649],"[{'name': 'default', 'title': 'Buy SiN Episode...","{'windows': True, 'mac': False, 'linux': False}","{'score': 75, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",{'total': 695},NaN,"{'coming_soon': False, 'date': '10 May, 2006'}","{'url': '', 'email': ''}","{'esrb': {'rating': 'm'}, 'dejus': {'rating_ge..."
236,game,"Sam & Max 103: The Mole, the Mob and the Meatball",8220,0.0,False,NaN,NaN,NaN,"English, French, German, Italian","{'minimum': 'Windows XP or Vista, 1.5GHz proce...",[],[],NaN,NaN,NaN,['Telltale Games'],['Skunkape Games'],NaN,NaN,NaN,[],"{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...",NaN,NaN,"{'coming_soon': False, 'date': '15 Jun, 2007'}","{'url': '', 'email': ''}",NaN
240,game,Sam & Max 106: Bright Side of the Moon,8250,0.0,False,NaN,NaN,NaN,"English, French, German, Italian","{'minimum': 'Windows XP or Vista, 1.5GHz proce...",[],[],NaN,NaN,NaN,['Telltale Games'],['Skunkape Games'],NaN,NaN,NaN,[],"{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...",NaN,NaN,"{'coming_soon': False, 'date': '15 Jun, 2007'}","{'url': '', 'email': ''}",NaN


# 2. Initial Process Data Function
- Targeted Functions: These functions will handle specific cleaning tasks for individual columns or data issues.
- process Function: This general-purpose function will execute all the defined cleaning functions on the DataFrame.
This modular approach allows for easy future modifications and ensures we're cleaning the correct data. We'll then run the process function and inspect the results.

In [10]:
def drop_null_cols(df, thresh=0.5):
    """Drop columns with more than a certain proportion of missing values (Default 50%)."""
    cutoff_count = len(df) * thresh
    
    return df.dropna(thresh=cutoff_count, axis=1)


def process_name_type(df):
    """Remove null values in name and type columns, and remove type column."""
    df = df[df['type'].notnull()]
    
    df = df[df['name'].notnull()]
    df = df[df['name'] != 'none']
    
    df = df.drop('type', axis=1)
    
    return df
    

def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""
    
    # Copy the input dataframe to avoid accidentally modifying original data
    df = df.copy()
    
    # Remove duplicate rows - all appids should be unique
    df = df.drop_duplicates()
    
    # Remove collumns with more than 50% null values
    df = drop_null_cols(df)
    
    # Process rest of columns
    df = process_name_type(df)
    
    return df

# Initial Data Shape
print('Initial DataSet: {}'.format(raw_steam_data.shape))

# After processing Data Shape
initial_processing = process(raw_steam_data)
print('DataSet after processing: {}'.format(initial_processing.shape))

# Columns Dropped
print('Columns to dropped: {}'.format(list(drop_rows)))
initial_processing.head()

Initial DataSet: (71193, 31)
DataSet after processing: (64786, 20)
Columns to dropped: ['controller_support', 'dlc', 'fullgame', 'legal_notice', 'drm_notice', 'ext_user_account_notice', 'demos', 'metacritic', 'reviews', 'recommendations']


,name,steam_appid,required_age,is_free,supported_languages,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,achievements,release_date,support_info,ratings
0,Counter-Strike,10,0.0,False,"English<strong>*</strong>, French<strong>*</st...",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'SGD', 'initial': 1000, 'final': ...","[574941, 7]","[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",{'usk': {'rating': '16'}}
1,Team Fortress Classic,20,0.0,False,"English, French, German, Italian, Spanish - Sp...",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'SGD', 'initial': 525, 'final': 5...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",NaN
2,Day of Defeat,30,0.0,False,"English, French, German, Italian, Spanish - Spain",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'SGD', 'initial': 525, 'final': 5...","[30, 944613]","[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",{'usk': {'rating': '16'}}
3,Deathmatch Classic,40,0.0,False,"English, French, German, Italian, Spanish - Sp...",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'SGD', 'initial': 525, 'final': 5...",[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '1 Jun, 2001'}","{'url': '', 'email': ''}",NaN
4,Half-Life: Opposing Force,50,0.0,False,"English, French, German, Korean",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Gearbox Software'],['Valve'],"{'currency': 'SGD', 'initial': 525, 'final': 5...",[32],"[{'name': 'default', 'title': 'Buy Half-Life: ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '1 Nov, 1999'}","{'url': 'https://help.steampowered.com', 'emai...",NaN


# 3. Data Processing

## 3.1 Processing `required_age` and `ratings` Column
### required_age
- Values stored as integers (0-20).
- Most enteries are 0, indicating low data varience
- The `process_age` function is created below to have an overall overview. <br>

Rows with 0 may mean they are unrated, unstated as in missing, or rated as suitable for everyone. 

### ratings
- The ratings column is used to provide parental guidance ratings.

To conclude for both columns, the birthday identification by steam is easily bypassed by lying about your age, thus we will be removing these 2 columns as they are both unreliable and irrelevant.

In [11]:
def process_age(df):
    # Convert 'required_age' column to numeric, handling errors by setting them to NaN
    df['required_age'] = pd.to_numeric(df['required_age'], errors='coerce')
    
    # Define bins and labels
    cut_points = [-1, 0, 3, 7, 12, 16, 2000]
    label_values = [0, 3, 7, 12, 16, 18]
    
    # Apply pd.cut function
    df['required_age'] = pd.cut(df['required_age'], bins=cut_points, labels=label_values)
    
    return df

# Example usage
age_df = process_age(initial_processing)
age_df['required_age'].value_counts().sort_index()

required_age
0     64348
3         5
7         5
12       35
16      124
18      266
Name: count, dtype: int64

## 3.2 Processing `platforms` and `requirements` Column
The `platforms` column holds 3 platform availability as boolean values (True/False) in a dictionary format. We'll convert this into 3 separate columns (one per platform) with the same boolean values stated **(True/False)**.
1. `Windows`
2. `Mac`
3. `Linux` <br>

The `requirements` column hold the specific requirements required by the user's desktop/laptop which is irrelevant in answering our question, therefore we will be dropping it. 

In [12]:
pd.DataFrame(age_df[['pc_requirements', 'mac_requirements', 'linux_requirements']]).head()

,pc_requirements,mac_requirements,linux_requirements
0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual..."
1,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual..."
2,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual..."
3,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual..."
4,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual..."


In [13]:
age_df['platforms'].head()

0    {'windows': True, 'mac': True, 'linux': True}
1    {'windows': True, 'mac': True, 'linux': True}
2    {'windows': True, 'mac': True, 'linux': True}
3    {'windows': True, 'mac': True, 'linux': True}
4    {'windows': True, 'mac': True, 'linux': True}
Name: platforms, dtype: object

In [14]:
platforms_first_row = age_df['platforms'].iloc[0]

print(type(platforms_first_row))

platforms_first_row

<class 'str'>


"{'windows': True, 'mac': True, 'linux': True}"

In [15]:
eval_first_row = literal_eval(platforms_first_row)

print(type(eval_first_row))
print(eval_first_row)

eval_first_row['windows']

<class 'dict'>
{'windows': True, 'mac': True, 'linux': True}


True

In [16]:
# create string of keys, joined on a semi-colon
';'.join(eval_first_row.keys())

'windows;mac;linux'

In [17]:
platforms = {'windows': True, 'mac': True, 'linux': False}

# list comprehension
print([x for x in platforms.keys() if platforms[x]])

# using list comprehension in join
';'.join(x for x in platforms.keys() if platforms[x])

['windows', 'mac']


'windows;mac'

Processing with `Series.apply`:

We can leverage pandas' `Series.apply` method to efficiently process each row in the "platforms" column. This method allows us to apply a custom function to each row, extracting the desired platform information.

In [18]:
def process_platforms(df):
    """Split platforms column into separate boolean columns for each platform."""
    # evaluate values in platforms column, so can index into dictionaries
    df = df.copy()
    
    def parse_platforms(x):
        
        d = literal_eval(x)
        
        return ';'.join(platform for platform in d.keys() if d[platform])
    
    df['platforms'] = df['platforms'].apply(parse_platforms)
    
    return df


platforms_df = process_platforms(age_df)
platforms_df['platforms'].value_counts()

platforms
windows              48278
windows;mac;linux     7557
windows;mac           6859
windows;linux         2068
linux                   12
mac                     11
mac;linux                1
Name: count, dtype: int64

## 3.3 Processing Price: `is_free` and `price_overview` Columns

The `is_free` column likely relates to the `price_overview` column (containing price information in a dictionary format). We will eventually combine these into a single column, where free games have a price of 0, changing price to a float type variable.

In [19]:
age_df['price_overview'].head()

0    {'currency': 'SGD', 'initial': 1000, 'final': ...
1    {'currency': 'SGD', 'initial': 525, 'final': 5...
2    {'currency': 'SGD', 'initial': 525, 'final': 5...
3    {'currency': 'SGD', 'initial': 525, 'final': 5...
4    {'currency': 'SGD', 'initial': 525, 'final': 5...
Name: price_overview, dtype: object

We'll check the null values under prices too, where there appears to be around 9000+ null in `price_overview`.

In [20]:
platforms_df['price_overview'].isnull().sum()

9491

In [21]:
not_free_and_null_price = platforms_df[(platforms_df['is_free'] == False) & (platforms_df['price_overview'].isnull())]
print('Number of Rows: {}'.format(not_free_and_null_price.shape[0]))
not_free_and_null_price.head()

Number of Rows: 3612


,name,steam_appid,required_age,is_free,supported_languages,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,achievements,release_date,support_info,ratings
14,Half-Life 2: Lost Coast,340,0,False,"English, French, German, Italian, Spanish - Sp...",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],['Valve'],['Valve'],NaN,NaN,[],windows;mac;linux,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '27 Oct, 2005'}","{'url': '', 'email': ''}",NaN
52,Final DOOM,2290,0,False,English,{'minimum': '<strong>Minimum: </strong>A 100% ...,[],[],['id Software'],['id Software'],NaN,NaN,[],windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '3 Aug, 2007'}","{'url': '', 'email': ''}","{'esrb': {'rating': 'm', 'descriptors': 'Blood..."
56,Quake II Mission Pack: The Reckoning,2330,0,False,English,{'minimum': '<strong>Minimum: </strong>A 100% ...,[],[],['Xatrix Entertainment'],['id Software'],NaN,NaN,[],windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '3 Aug, 2007'}","{'url': '', 'email': ''}","{'esrb': {'rating': 'm', 'descriptors': 'Blood..."
57,Quake II Mission Pack: Ground Zero,2340,0,False,English,{'minimum': '<strong>Minimum: </strong>A 100% ...,[],[],['Rogue Entertainment'],['id Software'],NaN,NaN,[],windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '3 Aug, 2007'}","{'url': '', 'email': ''}","{'esrb': {'rating': 'm', 'descriptors': 'Blood..."
62,The Ship: Single Player,2420,0,False,"English, French, German, Italian, Spanish - Sp...",{'minimum': '<strong>Minimum:</strong> 1.8 GHz...,[],[],['Outerlight Ltd.'],['Blazing Griffin Ltd.'],NaN,[56669],"[{'name': 'default', 'title': 'Buy The Ship: S...",windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...",NaN,"{'coming_soon': False, 'date': '20 Nov, 2006'}","{'url': '', 'email': ''}",NaN


In [22]:
platforms_df['price_overview'][0] #checking the first index/game

"{'currency': 'SGD', 'initial': 1000, 'final': 1000, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': 'S$10.00'}"

## 3.4 Processing Price Variations:

- Missing Values: We use `literal_eval` to convert price strings to dictionaries. If a value is missing, we return a dictionary with -1 for the initial value (allowing us to later set free games to 0 and easily identify truly missing prices).
- Extracting Currency and Price: We create separate `currency`and `price` columns using a lambda function to extract the corresponding values from the `price_overview` dictionary.

In [23]:
def process_price(df):
    df = df.copy()
        
    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {'currency': 'SGD', 'initial': -1}
    
    # evaluate as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(parse_price)
    
    # Create columns from currency and initial values
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])
    
    # Set price of free games to 0
    df.loc[df['is_free'], 'price'] = 0
    
    return df

price_data = process_price(platforms_df)[['name', 'currency', 'price']]
price_data.head()

,name,currency,price
0,Counter-Strike,SGD,1000
1,Team Fortress Classic,SGD,525
2,Day of Defeat,SGD,525
3,Deathmatch Classic,SGD,525
4,Half-Life: Opposing Force,SGD,525


Checking for games that don't have SGD listed as the currency.

In [24]:
price_data[price_data['currency'] == 'SGD'] #data whose prices are in SGD

,name,currency,price
0,Counter-Strike,SGD,1000
1,Team Fortress Classic,SGD,525
2,Day of Defeat,SGD,525
3,Deathmatch Classic,SGD,525
4,Half-Life: Opposing Force,SGD,525
...,...,...,...
71188,Windowkill,SGD,525
71189,SHIJIE XIUXIAN,SGD,215
71190,"Knowledge, or know Lady",SGD,900
71191,Area 19,SGD,1200


In [25]:
price_data[price_data['currency'] != 'SGD'] #data whose prices are NOT in SGD

,name,currency,price
6,Half-Life,EUR,819
517,Prince of Persia®,EUR,999
906,FINAL FANTASY XIV Online,USD,1999
1479,Tomb Raider,USD,1499
1517,Dishonored,EUR,999
...,...,...,...
67931,Beautiful Mystic Survivors,EUR,399
68072,Fleet Commander: Pacific,USD,2499
68294,Stellar Orphans,USD,1699
68394,No Son Of Mine,EUR,1249


In [26]:
not_in_sgd_games = price_data[price_data['currency'] != 'SGD']

def test_convert_to_sgd(row):
    # Convert prices to SGD if they are in foreign currencies
    exchange_rates = {'USD': 1.35, 'EUR': 1.50}  # Example exchange rates
    currency = row['currency']
    price = row['price']
    name = row['name']
    if currency != 'SGD':
        conversion_rate = exchange_rates.get(currency, 1)  # Default to 1 if currency not found
        converted_price = round(price * conversion_rate / 100, 2)  # Divide by 100 for better readability
        return f"{name}: {converted_price}"
    else:
        converted_price = round(price / 100, 2)  # Divide by 100 for better readability
        return f"{name}: {converted_price}"

# Apply the conversion function to each row of the not_in_sgd_games DataFrame and print the results
converted_prices = not_in_sgd_games.apply(test_convert_to_sgd, axis=1)
print(converted_prices)

6                        Half-Life: 12.29
517              Prince of Persia®: 14.98
906       FINAL FANTASY XIV Online: 26.99
1479                   Tomb Raider: 20.24
1517                    Dishonored: 14.98
                       ...               
67931    Beautiful Mystic Survivors: 5.99
68072     Fleet Commander: Pacific: 33.74
68294              Stellar Orphans: 22.94
68394               No Son Of Mine: 18.73
68881                    StreamWare: 8.09
Length: 225, dtype: object


Foreign Currencies can either:
- Convert them to SGD using exchange rates (complex).
- Removing these rows (simpler) <br>

We will be **converting** these games to SGD. It will not be wise to remove these games as certain games listed here are quite popular (Half-Life & FINAL FANTASY)

**Price Scaling:** We'll divide all prices by 100 to represent them as floats for better readability too.

In [27]:
def process_price(df):
    """Process price_overview column into formatted price column."""
    df = df.copy()
    
    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {'currency': 'SGD', 'initial': -1}
    
    # Evaluate as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(parse_price)
    
    # Create columns from currency and initial values
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])
    
    # Set price of free games to 0
    df.loc[df['is_free'], 'price'] = 0
    
    # Convert prices to SGD if they are in foreign currencies
    exchange_rates = {'USD': 1.35, 'EUR': 1.50}  # Example exchange rates
    
    def convert_to_sgd(row):
        currency = row['currency']
        price = row['price']
        if currency != 'SGD':
            conversion_rate = exchange_rates.get(currency, 1)  # Default to 1 if currency not found
            return round(price * conversion_rate / 100, 2)  # Divide by 100 for better readability
        else:
            return round(price / 100, 2)  # Divide by 100 for better readability
    
    df['price'] = df.apply(convert_to_sgd, axis=1)
    
    # Remove columns no longer needed
    df = df.drop(['is_free', 'currency', 'price_overview'], axis=1)
    
    return df


price_df = process_price(platforms_df)
price_df[['name', 'price']].head()

,name,price
0,Counter-Strike,10.00
1,Team Fortress Classic,5.25
2,Day of Defeat,5.25
3,Deathmatch Classic,5.25
4,Half-Life: Opposing Force,5.25


## 3.5 Processing Packages: `packages` and `package_groups` columns
- Packages and package_groups columns hold information about bundled games.
- We can ignore these columns (packages, package_groups) for price analysis since they don't directly affect price data.
- Our focus is on deciding what to do with rows missing price information.

In [28]:
# temporarily set a pandas option using with and option_context
with pd.option_context("display.max_colwidth", 500):
    display(price_df[['steam_appid', 'packages', 'package_groups', 'price']].head(3))

,steam_appid,packages,package_groups,price
0,10,"[574941, 7]","[{'name': 'default', 'title': 'Buy Counter-Strike', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 7, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Counter-Strike: Condition Zero - S$10.00', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 1000}, {'packageid': 574941, 'percent_savings_text': ' '...",10.00
1,20,[29],"[{'name': 'default', 'title': 'Buy Team Fortress Classic', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 29, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Team Fortress Classic - S$5.25', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 525}]}]",5.25
2,30,"[30, 944613]","[{'name': 'default', 'title': 'Buy Day of Defeat', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 30, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Day of Defeat - S$5.25', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 525}, {'packageid': 944613, 'percent_savings_text': ' ', 'percent_savings'...",5.25


- Rows without package information: These likely represent single games (not bundles).
- Rows with empty `package_groups`: These represent games in bundles, but the specific bundle information is missing

In [29]:
print('Null counts:', price_df['package_groups'].isnull().sum())
print('Empty list counts:', price_df[price_df['package_groups'] == "[]"].shape[0])

Null counts: 0
Empty list counts: 9187


In [30]:
missing_price_and_package = price_df[(price_df['price'] == -1) & (price_df['package_groups'] == "[]")]

print('Number of rows:', missing_price_and_package.shape[0], '\n')

Number of rows: 0 



Two missing price data types identified:
- Single games (no package info)
- Games in bundles with missing specific bundle details (empty package_groups)

Analyzed bundled games:
- May be superseded remasters, part of larger bundles, or episodic content.
- Extracting prices from bundles is complex.

Decision: 
- Remove these rows as the reasons behind the missing prices and bundle game are ambiguous.
- update `process_price` function to handle this logic.

In [31]:
def process_price(df):
    """Process price_overview column into formatted price column, and take care of package columns."""
    df = df.copy()
    
    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {'currency': 'SGD', 'initial': -1}
    
    # evaluate as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(parse_price)
    
    # create columns from currency and initial values
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])
    
    # set price of free games to 0
    df.loc[df['is_free'], 'price'] = 0
    
    # remove non-GBP rows
    df = df[df['currency'] == 'SGD']
    
    # remove rows where price is -1
    df = df[df['price'] != -1]
    
    # change price to display in pounds (can apply to all now -1 rows removed)
    df['price'] /= 100
    
    # remove columns no longer needed
    df = df.drop(['is_free', 'currency', 'price_overview', 'packages', 'package_groups'], axis=1)
    
    return df


price_df = process_price(platforms_df)
price_df.head()

,name,steam_appid,required_age,supported_languages,pc_requirements,mac_requirements,linux_requirements,developers,publishers,platforms,categories,genres,achievements,release_date,support_info,ratings,price
0,Counter-Strike,10,0,"English<strong>*</strong>, French<strong>*</st...",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",{'usk': {'rating': '16'}},10.00
1,Team Fortress Classic,20,0,"English, French, German, Italian, Spanish - Sp...",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",NaN,5.25
2,Day of Defeat,30,0,"English, French, German, Italian, Spanish - Spain",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",{'usk': {'rating': '16'}},5.25
3,Deathmatch Classic,40,0,"English, French, German, Italian, Spanish - Sp...",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '1 Jun, 2001'}","{'url': '', 'email': ''}",NaN,5.25
4,Half-Life: Opposing Force,50,0,"English, French, German, Korean",{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Gearbox Software'],['Valve'],windows;mac;linux,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '1 Nov, 1999'}","{'url': 'https://help.steampowered.com', 'emai...",NaN,5.25


## 3.6 Processing Languages: `supported_languages` column
We'll focus on games available in English. However, instead of removing non-English games, we'll create a new column with a boolean value (True/False) indicating English support.
- Identifying Missing Languages: We'll start by finding rows with missing language data.

In [32]:
print('Rows with Missing Languages: {}'.format(price_df['supported_languages'].isnull().sum()))
price_df[price_df['supported_languages'].isnull()]

Rows with Missing Languages: 10


,name,steam_appid,required_age,supported_languages,pc_requirements,mac_requirements,linux_requirements,developers,publishers,platforms,categories,genres,achievements,release_date,support_info,ratings,price
4962,Subsiege,338640,0,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],['Icebird Studios'],['Icebird Studios'],windows,"[{'id': 62, 'description': 'Family Sharing'}]",NaN,NaN,"{'coming_soon': False, 'date': '7 Sep, 2018'}","{'url': 'http://subsiege-game.com/', 'email': ...",NaN,18.50
14560,MARS VR(全球使命VR),596560,0,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,"['Ying Pei Digital Technology Shanghai Co., Li...","['SHANGHAI ZHENYOU TECHNOLOGY CO.,LTD']",windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '73', 'description': 'Violent'}, {'id'...",NaN,"{'coming_soon': False, 'date': '5 Apr, 2017'}","{'url': 'http://www.zygames.com/contact', 'ema...",NaN,3.50
22455,WIDECROSS,803800,0,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],['DEV TEAM'],['DEV TEAM'],windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...",NaN,"{'coming_soon': False, 'date': '26 Nov, 2019'}","{'url': '', 'email': ''}",NaN,7.50
22457,It Could Have Been Me,803840,0,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"['Åsa Egnér vr-designer', 'Alexander Cobleigh ...",['Kreativitetsbanken'],windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '23', 'description': 'Indie'}]",NaN,"{'coming_soon': False, 'date': '19 May, 2019'}","{'url': 'http://www.kreativitetsbanken.nu/', '...",NaN,3.25
27804,SNUSE 221,948070,0,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],['SNUSE GM'],['Conglomerate 5'],windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...",NaN,"{'coming_soon': False, 'date': '2 Apr, 2019'}","{'url': '', 'email': 'conglomerate5games@gmail...",NaN,1.10
27866,Dark Man,949720,0,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,[''],windows,"[{'id': 2, 'description': 'Single-player'}, {'...",NaN,NaN,"{'coming_soon': False, 'date': '8 May, 2020'}","{'url': '', 'email': ''}",{'dejus': {'required_age': '18'}},9.00
33359,Escape from Classroom,1103300,0,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],['jaremystudio'],['jaremystudio'],windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...",NaN,"{'coming_soon': False, 'date': '13 Jul, 2019'}","{'url': '', 'email': 'jaremystudio@gmail.com'}",NaN,4.25
36296,Aggressors,1184960,0,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],['HubiBoarStudio'],['HubiBoarStudio'],windows,"[{'id': 62, 'description': 'Family Sharing'}]","[{'id': '70', 'description': 'Early Access'}]",NaN,"{'coming_soon': False, 'date': '1 Feb, 2020'}","{'url': '', 'email': ''}",{'dejus': {'rating': 'l'}},5.25
54049,Sky DarkCrow,1785560,0,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],['Skyfox_13'],['Skyfox_13'],windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...",NaN,"{'coming_soon': False, 'date': '29 Nov, 2021'}","{'url': '', 'email': 'thales.steam2021@gmail.c...",{'dejus': {'rating': 'l'}},3.25
58089,Bitch demon twins,1957650,0,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],['DAI'],['DAI'],windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN,"{'coming_soon': False, 'date': '11 May, 2022'}","{'url': '', 'email': 'b094205188@gmail.com'}",NaN,1.10


Surprisingly, there doesn't seem to be anything missing. Or maybe rows with missing languages have been removed previously. Doing a manual scan on the csv, there doesn't appear a to be any missing data under `supported_languages`. (got 10 ^ tho wym by there isnt anything missing)

The `supported_languages` column holds language information in various formats:

- Comma-separated lists
- Mix of HTML and headings

Assuming English games will have "English" in the string, we'll use string searching to create a new boolean column indicating English support:

- True - "English" is found in the string.
- False - "English" is not found.

First, we will look at the general output. Since there are too many language combinations, we will just view the top 10 combinations/individual supported langugues.

In [33]:
print(price_df['supported_languages'].iloc[0])
price_df['supported_languages'].value_counts().head(10)

English<strong>*</strong>, French<strong>*</strong>, German<strong>*</strong>, Italian<strong>*</strong>, Spanish - Spain<strong>*</strong>, Simplified Chinese<strong>*</strong>, Traditional Chinese<strong>*</strong>, Korean<strong>*</strong><br><strong>*</strong>languages with full audio support


supported_languages
English                                                                                                        16572
English<strong>*</strong><br><strong>*</strong>languages with full audio support                               14488
English, Russian                                                                                                1221
English, Simplified Chinese                                                                                      778
English, Japanese                                                                                                706
Simplified Chinese                                                                                               636
Simplified Chinese<strong>*</strong><br><strong>*</strong>languages with full audio support                      458
English<strong>*</strong>, Russian<strong>*</strong><br><strong>*</strong>languages with full audio support      363
English, Portuguese - Brazil                

We'll leverage `Series.apply` to efficiently process each row in the `supported_languages` column. 
- This method allows us to apply a custom function that checks for the presence of the string `english` (case-insensitive). 
- The function returns 1 if found (interpreted as True) and 0 otherwise (interpreted as False). This approach saves space in the CSV file.
- We apply the `lower()` string method so capitalisation doesn't matter.

In [34]:
def process_language(df):
    """Process supported_languages column into a boolean 'is english' column."""
    df = df.copy()
    
    # drop rows with missing language data
    df = df.dropna(subset=['supported_languages'])
    
    df['english'] = df['supported_languages'].apply(lambda x: 1 if 'english' in x.lower() else 0)
    df = df.drop('supported_languages', axis=1)
    
    return df


language_df = process_language(price_df)
language_df[['name', 'english']].head()

,name,english
0,Counter-Strike,1
1,Team Fortress Classic,1
2,Day of Defeat,1
3,Deathmatch Classic,1
4,Half-Life: Opposing Force,1


In [35]:
language_df['english'].value_counts()

english
1    58946
0     1993
Name: count, dtype: int64

## 3.7 Processing `developers` and `publishers` Column
- First, we check for null values:

In [36]:
print('Developers null counts:', language_df['developers'].isnull().sum())
print('Developers empty list counts:', language_df[language_df['developers'] == "['']"].shape[0])

print('\nPublishers null counts:', language_df['publishers'].isnull().sum())
print('Publishers empty list counts:', language_df[language_df['publishers'] == "['']"].shape[0])

Developers null counts: 111
Developers empty list counts: 0

Publishers null counts: 0
Publishers empty list counts: 323


Some possible solutions we thought to handle list-type columns:
- Multiple Columns: Create separate columns for each potential value (e.g., `developer_1`, `developer_2`). (May become cumbersome for variable list lengths).
- First Value + Other Values: Separate columns for first value (e.g., `developer_1`) and combined remaining values (e.g., `other_developers`). (Simpler but discards information).
- First Value Only: Create a column with just the first value, discarding the rest. (Simplest but loses information).
- Combined Values: Combine all list elements into a single column (e.g., "`developer1`, `developer2`"). (Preserves information but requires further processing).

Choosing the best approach:
- Analyzing the data reveals that most rows have only one value in the list-type columns (developers, publishers).
- Creating multiple columns (option 1 & 2) or using only the first value (option 3) would lead to mostly missing data, defeating the purpose.
Therefore, the best approach is option 4: Combine all values into a single column.

- This method uses `str.join()` to combine list elements into a single string separated by a delimiter (e.g., comma).
- It preserves information from both single and multiple value entries.

Benefits:
- No missing data introduced.
- Maintains information about multiple developers/publishers.

Next step:
- Implement this approach using `str.join()` to create new columns with combined developer and publisher information.

In [37]:
def process_developers_and_publishers(df):
    """Parse columns as semicolon-separated string."""
    # remove rows with missing data (~ means not)
    df = df[(df['developers'].notnull()) & (df['publishers'] != "['']")].copy()
    df = df[~(df['developers'].str.contains(';')) & ~(df['publishers'].str.contains(';'))]
    df = df[(df['publishers'] != "['NA']") & (df['publishers'] != "['N/A']")]
    
    # create list for each
    df['developer'] = df['developers'].apply(lambda x: ';'.join(literal_eval(x)))
    df['publisher'] = df['publishers'].apply(lambda x: ';'.join(literal_eval(x)))

    df = df.drop(['developers', 'publishers'], axis=1)
    
    return df

dev_pub_df = process_developers_and_publishers(language_df)
dev_pub_df[['name', 'steam_appid', 'developer', 'publisher']].head()

,name,steam_appid,developer,publisher
0,Counter-Strike,10,Valve,Valve
1,Team Fortress Classic,20,Valve,Valve
2,Day of Defeat,30,Valve,Valve
3,Deathmatch Classic,40,Valve,Valve
4,Half-Life: Opposing Force,50,Gearbox Software,Valve


## 3.8 Processing Categories and Genres
Both "categories" and "genres" columns seem to follow a similar structure:
- List of dictionaries
- Each dictionary contains "id" and "description" key-value pairs

We'll likely extract just the descriptions into separate lists for now.

In [38]:
print('Categories:\n')
print('Null values:', price_df['categories'].isnull().sum())
print()
print(price_df['categories'][0])

print('\nGenres:\n')
print('Null values:', price_df['genres'].isnull().sum())
print()
print(price_df['genres'].iloc[0])

Categories:

Null values: 603

[{'id': 1, 'description': 'Multi-player'}, {'id': 49, 'description': 'PvP'}, {'id': 36, 'description': 'Online PvP'}, {'id': 37, 'description': 'Shared/Split Screen PvP'}, {'id': 8, 'description': 'Valve Anti-Cheat enabled'}, {'id': 62, 'description': 'Family Sharing'}]

Genres:

Null values: 101

[{'id': '1', 'description': 'Action'}]


In [39]:
def process_categories_and_genres(df):
    df = df.copy()
    df = df[(df['categories'].notnull()) & (df['genres'].notnull())]
    
    for col in ['categories', 'genres']:
        df[col] = df[col].apply(lambda x: ';'.join(item['description'] for item in literal_eval(x)))
    
    return df

cat_gen_df = process_categories_and_genres(price_df)
cat_gen_df[['steam_appid', 'categories', 'genres']].head()

,steam_appid,categories,genres
0,10,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
1,20,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
2,30,Multi-player;Valve Anti-Cheat enabled;Family S...,Action
3,40,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
4,50,Single-player;Multi-player;Valve Anti-Cheat en...,Action


## 3.9 Processing `release_date` column
- currently, the release date format looks difficult to read
- Example: {'coming_soon': False, 'date': '1 Nov, 2000'}
- We shall process it to look something like: 2000-11-01 (better readability!)

In [42]:
def process_release_date(df):
    df = df.copy()
    
    def eval_date(x):
        x = literal_eval(x)
        if x['coming_soon']:
            return '' # return blank string so can drop missing at end
        else:
            return x['date']
    
    df['release_date'] = df['release_date'].apply(eval_date)
    
    def parse_date(x):
        if re.search(r'[\d]{1,2} [A-Za-z]{3}, [\d]{4}', x):
            return x.replace(',', '')
        elif re.search(r'[A-Za-z]{3} [\d]{4}', x):
            return '1 ' + x
        elif x == '':
            return np.nan
        else:
            return None
            
    df['release_date'] = df['release_date'].apply(parse_date)
    df['release_date'] = pd.to_datetime(df['release_date'], format='%d %b %Y', errors='coerce')
    df['release_date'] = df['release_date'].replace('/','-') #added
    
    df = df[df['release_date'].notnull()]
    
    return df


# 4. Exporting Cleaned Data
Function Update: Include logic in the process function to handle both single and multiple values in the developer and publisher columns.

Use `str.join()` to combine list elements into a single string (e.g., comma-separated).

Run Cleaning: Apply the updated function to the entire dataset. Create a new dataframe with the cleaned rows (combined developer/publisher information).

As per our previous reasonings we would be dropping these columns here <br>
**Columns to Drop:** ['required_age', 'pc_requirements', 'mac_requirements', 'linux_requirements', 'support_info', 'ratings']

In [43]:
def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""
    
    # Copy the input dataframe to avoid accidentally modifying original data
    df = df.copy()
    
    # Remove duplicate rows - all appids should be unique
    df = df.drop_duplicates()
    
    # Remove collumns with more than 50% null values
    df = drop_null_cols(df)
    
    # Process columns
    df = process_name_type(df)
    df = process_age(df)
    df = process_platforms(df)
    df = process_price(df)
    df = process_language(df)
    df = process_developers_and_publishers(df)
    df = process_categories_and_genres(df) 
    df = process_release_date(df)
    

    
    return df

steam_data = process(raw_steam_data)
steam_data = steam_data.drop(['required_age', 'pc_requirements', 'mac_requirements', 'linux_requirements', 'support_info', 'ratings']
                             , axis=1)
steam_data.head()

,name,steam_appid,platforms,categories,genres,achievements,release_date,price,english,developer,publisher
0,Counter-Strike,10,windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,NaN,2000-11-01,10.00,1,Valve,Valve
1,Team Fortress Classic,20,windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,NaN,1999-04-01,5.25,1,Valve,Valve
2,Day of Defeat,30,windows;mac;linux,Multi-player;Valve Anti-Cheat enabled;Family S...,Action,NaN,2003-05-01,5.25,1,Valve,Valve
3,Deathmatch Classic,40,windows;mac;linux,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,NaN,2001-06-01,5.25,1,Valve,Valve
4,Half-Life: Opposing Force,50,windows;mac;linux,Single-player;Multi-player;Valve Anti-Cheat en...,Action,NaN,1999-11-01,5.25,1,Gearbox Software,Valve


In [44]:
steam_data.to_csv('Cleaned\steam_data_cleaned.csv', index=False)